In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [2]:
# Reading ratings file
# train_ratings = pd.read_excel("/content/drive/MyDrive/test_data.xlsx", usecols=['userId','movieId','rating','timestamp_ratings'])

# Reading movies file
# train_movies = pd.read_excel("/content/drive/MyDrive/test_data.xlsx", usecols=['movieId','title','genres_x'])

In [3]:

df_movies = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['movieId','title','genres_x'])
df_ratings = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['userId','movieId','rating','timestamp'])

In [4]:
df_movies

,movieId,title,genres_x
0,2249,My Blue Heaven,Comedy
1,160438,Jason Bourne,Action
2,8810,AVP: Alien vs. Predator,Action|Horror|Sci-Fi|Thriller
3,2018,Bambi,Animation|Children|Drama
4,1645,The Devil's Advocate,Drama|Mystery|Thriller
...,...,...,...
286359,139385,The Revenant,Adventure|Drama
286360,90866,Hugo,Children|Drama|Mystery
286361,8169,*batteries not included,Children|Comedy|Fantasy|Sci-Fi
286362,1292,Being There,Comedy|Drama


In [5]:
df_ratings

,userId,movieId,rating,timestamp
0,126180,2249,2.5,1150714271
1,40636,160438,2.0,1553878417
2,47628,8810,3.5,1107559253
3,109731,2018,4.0,1025848511
4,93451,1645,3.5,1121352601
...,...,...,...,...
286359,49403,139385,2.5,1505950963
286360,137293,90866,3.0,1330138811
286361,81216,8169,3.0,1551306328
286362,111258,1292,3.5,1544936052


In [6]:
df_ratings.userId=df_ratings.userId.apply(lambda x: str())

In [7]:
type(df_ratings.userId)

pandas.core.series.Series

In [8]:
df_movies_ratings = pd.read_csv(r"C:\Users\Hazel Tan\Documents\BT4222\datasets_BT4222\train_data.csv", usecols=['movieId','title','genres_x','userId','rating','timestamp'])
df_movies_ratings

,userId,movieId,rating,timestamp,title,genres_x
0,126180,2249,2.5,1150714271,My Blue Heaven,Comedy
1,40636,160438,2.0,1553878417,Jason Bourne,Action
2,47628,8810,3.5,1107559253,AVP: Alien vs. Predator,Action|Horror|Sci-Fi|Thriller
3,109731,2018,4.0,1025848511,Bambi,Animation|Children|Drama
4,93451,1645,3.5,1121352601,The Devil's Advocate,Drama|Mystery|Thriller
...,...,...,...,...,...,...
286359,49403,139385,2.5,1505950963,The Revenant,Adventure|Drama
286360,137293,90866,3.0,1330138811,Hugo,Children|Drama|Mystery
286361,81216,8169,3.0,1551306328,*batteries not included,Children|Comedy|Fantasy|Sci-Fi
286362,111258,1292,3.5,1544936052,Being There,Comedy|Drama


In [9]:
ratings_matrix_items = df_movies_ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix_items.fillna( 0, inplace = True )
ratings_matrix_items.shape

(2073, 188)

In [10]:
ratings_matrix_items

userId,1748,1920,2177,2982,3150,8619,9620,11952,14831,17474,...,150005,151214,152107,152690,156183,156480,159816,160747,162047,162516
0,4.5,3.5,0.0,0.0,3.0,0.0,4.0,4.5,4.5,0.0,...,0.0,3.0,5.0,3.0,5.0,4.0,4.0,4.0,3.0,4.5
1,3.0,3.5,3.0,2.0,4.0,2.5,4.0,3.5,0.0,4.0,...,2.5,1.0,3.0,0.0,3.5,0.0,3.0,0.0,0.0,2.5
2,2.5,0.0,0.0,0.0,0.0,2.5,3.0,1.0,3.0,3.0,...,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.5,0.0
3,0.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4.0,0.0,3.0,2.5,0.0,2.0,0.0,3.0,0.0,4.5,...,0.0,0.0,0.5,0.0,3.0,0.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
2069,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2070,4.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5


In [11]:
ratings_matrix_items.sum()

userId
1748      3364.5
1920      3067.5
2177      3959.5
2982      3194.0
3150      3109.0
           ...  
156480    2846.5
159816    3783.5
160747    3169.0
162047    3402.0
162516    4673.5
Length: 188, dtype: float64

In [12]:
movie_similarity = 1 - pairwise_distances( ratings_matrix_items.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_items = pd.DataFrame( movie_similarity )
ratings_matrix_items

,0,1,2,3,4,5,6,7,8,9,...,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072
0,0.000000,0.695623,0.591881,0.330018,0.630691,0.795862,0.661242,0.499564,0.729747,0.479992,...,0.383551,0.456254,0.414543,0.443204,0.347393,0.405858,0.452568,0.380845,0.323666,0.452285
1,0.695623,0.000000,0.533492,0.387662,0.577920,0.722715,0.636219,0.485175,0.692693,0.519221,...,0.362524,0.422776,0.390899,0.419694,0.328172,0.385526,0.365462,0.428565,0.362037,0.389286
2,0.591881,0.533492,0.000000,0.239740,0.587282,0.623924,0.574958,0.395930,0.525417,0.431209,...,0.384352,0.412509,0.412058,0.344635,0.363017,0.396150,0.358258,0.367960,0.238942,0.375609
3,0.330018,0.387662,0.239740,0.000000,0.413093,0.391573,0.439768,0.286466,0.329965,0.215487,...,0.139556,0.211740,0.126030,0.143463,0.082265,0.159238,0.115081,0.161901,0.102023,0.120741
4,0.630691,0.577920,0.587282,0.413093,0.000000,0.615366,0.612777,0.400400,0.563991,0.411772,...,0.253594,0.335475,0.258237,0.370468,0.263779,0.311165,0.311294,0.312416,0.214890,0.264940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,0.405858,0.385526,0.396150,0.159238,0.311165,0.457305,0.386618,0.245531,0.341090,0.309421,...,0.542234,0.668254,0.604577,0.640705,0.627792,0.000000,0.613166,0.491842,0.450310,0.711105
2069,0.452568,0.365462,0.358258,0.115081,0.311294,0.418271,0.345826,0.218629,0.407484,0.254890,...,0.608061,0.662952,0.587389,0.685184,0.506171,0.613166,0.000000,0.562648,0.420752,0.633178
2070,0.380845,0.428565,0.367960,0.161901,0.312416,0.404633,0.323210,0.230151,0.361691,0.365119,...,0.473164,0.597104,0.564075,0.575960,0.486494,0.491842,0.562648,0.000000,0.482566,0.552018
2071,0.323666,0.362037,0.238942,0.102023,0.214890,0.340743,0.337283,0.128454,0.294411,0.249466,...,0.501266,0.532236,0.420518,0.437234,0.321120,0.450310,0.420752,0.482566,0.000000,0.460344


In [13]:
def item_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=df_movies[df_movies['title']==user_inp].index.tolist()
        inp=inp[0]

        df_movies['similarity'] = ratings_matrix_items.iloc[inp]
        df_movies.columns = ['movie_id', 'title', 'release_date','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [14]:
def recommendedMoviesAsperItemSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin([5.0,4.5])][['title']]
    # print(user_movie)
    user_movie=user_movie.iloc[0,0]
    # print(user_movie)
    item_similarity(user_movie)
    sorted_movies_as_per_userChoice=df_movies.sort_values( ["similarity"], ascending = False )
    # print(sorted_movies_as_per_userChoice)
    sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45]['movie_id']
    # print(sorted_movies_as_per_userChoice)
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= df_ratings[df_ratings['userId']== user_id]['movieId']
    # print(user2Movies)
    for movieId in sorted_movies_as_per_userChoice:
            # print(movieId)
            if movieId not in user2Movies:
                # print(movieId)
                df_new= df_ratings[(df_ratings.movieId==movieId)]
                # print(df_new)
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

In [15]:
def movieIdToTitle(listMovieIDs):
    """
     Converting movieId to titles
    :param user_id: List of movies
    :return: movie titles
    """
    movie_titles= set()
    for id in listMovieIDs:
        movie_titles.add(df_movies[df_movies['movie_id']==id]['title'].unique()[0])
    return movie_titles

In [16]:
user_id=1748
movieIdToTitle(recommendedMoviesAsperItemSimilarity(user_id))  

{'Dancer in the Dark', 'Superman', 'Taxi Driver'}

In [17]:
ratings_matrix_users = df_movies_ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating').reset_index(drop=True)
ratings_matrix_users.fillna( 0, inplace = True )
movie_similarity = 1 - pairwise_distances( ratings_matrix_users.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_users = pd.DataFrame( movie_similarity )
ratings_matrix_users

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.000000,0.548267,0.568642,0.504554,0.566822,0.532100,0.596568,0.553005,0.525475,0.471402,...,0.539998,0.500210,0.555773,0.578303,0.505709,0.449474,0.563255,0.507075,0.516558,0.611842
1,0.548267,0.000000,0.488204,0.477488,0.608433,0.509052,0.540724,0.564231,0.443346,0.622190,...,0.494715,0.430610,0.541512,0.542820,0.555267,0.483651,0.537983,0.544152,0.578244,0.627185
2,0.568642,0.488204,0.000000,0.640191,0.455358,0.672703,0.646190,0.558346,0.647440,0.428688,...,0.624512,0.642085,0.647140,0.647411,0.433148,0.395275,0.614695,0.535773,0.525580,0.611026
3,0.504554,0.477488,0.640191,0.000000,0.419785,0.639102,0.552249,0.539383,0.624760,0.428400,...,0.577657,0.650748,0.612875,0.629929,0.410693,0.404565,0.551264,0.540558,0.503595,0.621715
4,0.566822,0.608433,0.455358,0.419785,0.000000,0.451348,0.506919,0.544528,0.411650,0.548933,...,0.478945,0.399301,0.478557,0.492984,0.555632,0.479881,0.531303,0.478473,0.541056,0.577325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.449474,0.483651,0.395275,0.404565,0.479881,0.461894,0.455206,0.395580,0.460316,0.424640,...,0.494015,0.384892,0.483838,0.461884,0.510710,0.000000,0.468626,0.563018,0.522645,0.566384
184,0.563255,0.537983,0.614695,0.551264,0.531303,0.624083,0.600376,0.558692,0.614050,0.475970,...,0.602889,0.553407,0.641138,0.626418,0.520271,0.468626,0.000000,0.571250,0.590637,0.625502
185,0.507075,0.544152,0.535773,0.540558,0.478473,0.603931,0.594778,0.472457,0.577156,0.471989,...,0.606020,0.518571,0.620349,0.568480,0.493944,0.563018,0.571250,0.000000,0.605166,0.647012
186,0.516558,0.578244,0.525580,0.503595,0.541056,0.586468,0.544654,0.494579,0.542272,0.482926,...,0.595030,0.508131,0.591377,0.580451,0.557566,0.522645,0.590637,0.605166,0.000000,0.639598


In [18]:
similar_user_series= ratings_matrix_users.idxmax(axis=1)
df_similar_user= similar_user_series.to_frame()

In [19]:
df_similar_user.columns=['similarUser']

In [20]:
df_similar_user

,similarUser
0,110
1,32
2,98
3,68
4,32
...,...
183,89
184,16
185,89
186,114


In [21]:
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [22]:
def user_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=df_movies_ratings[df_movies_ratings['title']==user_inp].index.tolist()
        inp=inp[0]
        # print(inp)

        df_ratings['similarity'] = ratings_matrix_users.iloc[inp]
        
        df_ratings.columns = ['userId', 'movieId', 'rating','timestamp','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [23]:
def getRecommendedMoviesAsperUserSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    # user2Movies= df_ratings[df_ratings['userId']== userId]['movieId']
    user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin([5.0,4.5])][['title']]
    # print(user_movie)
    # sim_user=df_similar_user.iloc[0,0]
    user_movie=user_movie.iloc[0,0]
    # print(user_movie)
    user_similarity(user_movie)
    sorted_users_as_per_movieChoice=df_movies.sort_values( ["similarity"], ascending = False )
    # print(sorted_movies_as_per_userChoice)
    sorted_users_as_per_movieChoice=sorted_users_as_per_movieChoice[sorted_users_as_per_movieChoice['similarity'] >=0.45]['movie_id']
    # print(sorted_movies_as_per_userChoice)
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= df_ratings[df_ratings['userId']== user_id]['movieId']
    # print(user2Movies)
    for movieId in sorted_users_as_per_movieChoice:
            # print(movieId)
            if movieId not in user2Movies:
                # print(movieId)
                df_new= df_ratings[(df_ratings.movieId==movieId)]
                # df_new= df_movies_ratings[(df_movies_ratings.userId==sim_user) & (df_movies_ratings.movieId==movieId)]
                # print(df_new)
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

In [24]:
# movieId_recommended=list()
# def getRecommendedMoviesAsperUserSimilarity(userId):
#     """
#      Recommending movies which user hasn't watched as per User Similarity
#     :param user_id: user_id to whom movie needs to be recommended
#     :return: movieIds to user 
#     """
#     user2Movies= df_ratings[df_ratings['userId']== userId]['movieId']
#     print(user2Movies)
#     sim_user=df_similar_user.iloc[0,0]
#     print(sim_user)
#     df_recommended=pd.DataFrame(columns=['movieId','title','genres','userId','rating','timestamp'])
#     for movieId in df_ratings[df_ratings['userId']== sim_user]['movieId']:
#         if movieId not in user2Movies:
#             df_new= df_movies_ratings[(df_movies_ratings.userId==sim_user) & (df_movies_ratings.movieId==movieId)]
#             df_recommended=pd.concat([df_recommended,df_new])
#         best10=df_recommended.sort_values(['rating'], ascending = False )[1:10]  
#     return best10['movieId']

In [25]:
user_id=1748
recommend_movies= movieIdToTitle(getRecommendedMoviesAsperUserSimilarity(user_id))
print("Movies you should watch are:\n")
print(recommend_movies)

Movies you should watch are:

{'Taxi Driver', 'Dancer in the Dark', 'Superman'}


In [26]:
def get_user_similar_movies( user1, user2 ):
    
    """
     Returning common movies and ratings of same for both the users
    :param user1,user2: user ids of 2 users need to compare
    :return: movieIds to user 
    """
    common_movies = df_movies_ratings[df_movies_ratings.userId == user1].merge(
      df_movies_ratings[df_movies_ratings.userId == user2],
      on = "movieId",
      how = "inner" )
    common_movies.drop(['movieId','genres_x_x','genres_x_y', 'timestamp_x','timestamp_y','title_y'],axis=1,inplace=True)
    return common_movies

In [27]:
get_user_similar_movies(1748,1920)

,userId_x,rating_x,title_x,userId_y,rating_y
0,1748,3.0,Inside Out,1920,4.0
1,1748,3.0,Inside Out,1920,4.0
2,1748,3.0,Inside Out,1920,4.0
3,1748,3.0,Inside Out,1920,4.0
4,1748,3.0,Inside Out,1920,4.0
...,...,...,...,...,...
2172,1748,4.5,Good Will Hunting,1920,4.0
2173,1748,4.0,Braveheart,1920,3.5
2174,1748,4.5,Kingsman: The Secret Service,1920,3.5
2175,1748,3.0,Nick of Time,1920,3.0
